# Intro & Loading 

In [1]:
from __future__ import print_function
import pandas as pd

from genepy.utils import helper as h
from gsheets import Sheets
from taigapy import TaigaClient

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have JKBio in your path:
e.g. have installed JKBio in the same folder as ccle_processing


/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

In [2]:
samplesetname = "21Q1"

virtual= {}
virtual['public'] = 'public-21q1-4b39'
virtual['ibm'] = 'ibm-21q1-abd9'
virtual['dmc'] = 'dmc-21q1-0e11'
virtual['internal']='internal-21q1-4fc4'


taiga_mutation = {}
taiga_mutation['internal'] = "depmap-mutation-calls-9be3"
taiga_mutation['ibm'] = "mutations-b05c"
taiga_mutation['dmc'] = "depmap-mutation-calls-dfce"
taiga_mutation['public'] ="depmap-mutation-calls-9a1a"

taiga_expression = {}
taiga_expression['internal'] ="depmap-rnaseq-expression-data-363a"
taiga_expression['ibm'] = "expression-9d97"
taiga_expression['dmc'] = "depmap-rnaseq-expression-data-80ef"
taiga_expression['public'] ="depmap-rnaseq-expression-data-ccd0"

taiga_fusion = {}
taiga_fusion['internal'] = "gene-fusions-8b7a"
taiga_fusion['ibm'] = "fusion-dc08"
taiga_fusion['dmc'] = "gene-fusions-375f"
taiga_fusion['public'] ="gene-fusions-6212"

taiga_copynumber = {}
taiga_copynumber['internal'] ="depmap-wes-cn-data-81a7"
taiga_copynumber['ibm'] = "cn-e20f"
taiga_copynumber['dmc'] = "depmap-cn-data-9b9d"
taiga_copynumber['public'] = "depmap-wes-cn-data-97cc"

prev_virtual = {}
#prev_virtual['public'] = 'public-20q3-3d35'
#prev_virtual['dmc'] = 'dmc-20q3-deprecated-never-released--5f55'
#prev_virtual['internal'] = 'internal-20q3-00d0'

prev_virtual['public'] = 'public-20q4-a4b3'
prev_virtual['dmc'] = 'dmc-20q4-fcf4'
# prev_virtual['ibm'] = 'ibm-20q4-269f'
prev_virtual['internal'] = 'internal-20q4-2540'
potential_list_url = "https://docs.google.com/spreadsheets/d/1YuKEgZ1pFKRYzydvncQt9Y_BKToPlHP-oDB-0CAv3gE"

release = samplesetname.lower()

In [55]:
new = {}
gsheets = sheets.get(potential_list_url).sheets[0].to_frame()
new['internal'] = set([i for i in gsheets['Internal'].values.tolist() if str(i) != "nan"])
new['dmc'] = set([i for i in gsheets['DMC'].values.tolist() if str(i) != "nan"])
new['ibm'] = set([i for i in gsheets['IBM'].values.tolist() if str(i) != "nan"])
new['public'] = set([i for i in gsheets['Public'].values.tolist() if str(i) != "nan"])


new["internal"] = new["internal"] | new["ibm"] | new["dmc"] | new["public"]
new["ibm"] = new["ibm"] | new["dmc"] | new["public"]
new["dmc"] = new["dmc"] | new["public"]

## Getting what was released before

In [56]:
prevmut = {}
prevrna = {}
prevcn = {}
prevwes = {}
prev = {}
for val in ['internal', 'dmc', 'public']:
    print(val)
    prevmut[val] = set(tc.get(name=prev_virtual[val], file='CCLE_mutations').DepMap_ID)
    prevrna[val] = set(tc.get(name=prev_virtual[val], file='CCLE_expression').index)
    prevcn[val] = set(tc.get(name=prev_virtual[val], file='CCLE_segment_cn').DepMap_ID)
    prev[val] = prevmut[val] | prevrna[val] | prevcn[val]
    prevwes[val] = prevmut[val] | prevcn[val]

internal
dmc
public


In [57]:
prevmut["dmc"] = prevmut["dmc"] | prevmut["public"]
prevrna["dmc"] = prevrna["dmc"] | prevrna["public"]
prevcn["dmc"] = prevcn["dmc"] | prevcn["public"]
prev["dmc"] = prev["dmc"] | prev["public"]
prevwes["dmc"] = prevwes["dmc"] | prevwes["public"]

prevmut["internal"] = prevmut["internal"] | prevmut["dmc"]
prevrna["internal"] = prevrna["internal"] | prevrna["dmc"]
prevcn["internal"] = prevcn["internal"] | prevcn["dmc"]
prev["internal"] = prev["internal"] | prev["dmc"]
prevwes["internal"] = prevwes["internal"] | prevwes["dmc"]

In [61]:
print('in cn but not mut')
print(prevwes["internal"] - prevmut["internal"])
print('in mut but not cn')
print(prevwes["internal"] - prevcn["internal"])
print('in rna but no wes/wgs')
print(prev["internal"] - prevwes["internal"])
print('in wes/wgs but not rna')
print(prev["internal"] - prevrna["internal"])

in cn but not mut
{'ACH-001175', 'ACH-001037', 'ACH-001198', 'ACH-001225', 'ACH-001171', 'ACH-001079', 'ACH-002335', 'ACH-001045', 'ACH-001088', 'ACH-001087', 'ACH-001071', 'ACH-001015', 'ACH-001224', 'ACH-001018', 'ACH-001000', 'ACH-001101', 'ACH-002303', 'ACH-001017', 'ACH-001249', 'ACH-001121'}
in mut but not cn
{'ACH-002014', 'ACH-001971', 'ACH-000033', 'ACH-001854', 'ACH-001187', 'ACH-002512', 'ACH-002394', 'ACH-002392', 'ACH-001349', 'ACH-001293', 'ACH-002396', 'ACH-000629', 'ACH-002395', 'ACH-001693', 'ACH-001210', 'ACH-001449', 'ACH-002010', 'ACH-001855', 'ACH-002390', 'ACH-001493', 'ACH-001438', 'ACH-001537', 'ACH-000712', 'ACH-001669', 'ACH-002393', 'ACH-001696', 'ACH-001131', 'ACH-001078', 'ACH-001676', 'ACH-002391', 'ACH-001662', 'ACH-001502', 'ACH-000084', 'ACH-001437', 'ACH-001108', 'ACH-000010', 'ACH-001011', 'ACH-001109', 'ACH-001672'}
in rna but no wes/wgs
{'ACH-001743', 'ACH-001429', 'ACH-001712', 'ACH-001316', 'ACH-001708', 'ACH-001393', 'ACH-001741'}
in wes/wgs but 

In [62]:
prev['ibm'] = prev['dmc']
prevcn['ibm'] = prevcn['dmc']
prevmut['ibm'] = prevmut['dmc']
prevrna['ibm'] = prevrna['dmc']
prevwes['ibm'] = prevwes['dmc']

## managing the readmes

In [6]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

fatal: destination path 'depmap-release-readmes' already exists and is not an empty directory.


In [134]:
! cd ../depmap-release-readmes && git pull

Updating bb67687..fa199c0
Fast-forward
 release-20q4/dmc-20q4.txt      | 41 +---------------------
 release-20q4/internal-20q4.txt | 77 ++++++++++++++++++++----------------------
 release-20q4/public-20q4.txt   | 74 ++++++++++++++++++++--------------------
 3 files changed, 74 insertions(+), 118 deletions(-)


In [143]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

Making public
Making internal
Making dmc
[master 87f7b1c] 21Q1
 4 files changed, 140 insertions(+), 454 deletions(-)
 delete mode 100644 release-21q1/ibm-21q1.txt
Counting objects: 6, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 2.63 KiB | 0 bytes/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/broadinstitute/depmap-release-readmes.git
   8cb35f2..87f7b1c  master -> master


In [ ]:
# NOW UPDATE THE READMEs

In [144]:
! mkdir temp/README/

mkdir: cannot create directory ‘temp/README/’: File exists


In [107]:
! cd ../depmap-release-readmes && cp release-$release/* ../ccle_processing/readmes/ && git add . && git commit -m "Omics: updating readmes to new release" && git push

On branch master
Your branch is ahead of 'origin/master' by 3 commits.
  (use "git push" to publish your local commits)
nothing to commit, working tree clean


In [106]:
! cd ../depmap-release-readmes && git pull &&  cp release-$release/* ../ccle_processing/readmes/

Auto-merging release-21q1/public-21q1.txt
7=▽  ]11;?"~/depmap-release-readmes/.git/MERGE_MSG" 7L, 309C[>cMerge branch 'master' of https://github.com/broadinstitute/depmap-release-readmees

# Please enter a commit message to explain why this merge is necessary,
# especially if it merges an updated upstream into a topic branch.
#
# Lines starting with '#' will be ignored, and an empty message aborts
# the commit.
~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                   

In [ ]:
for val in ['internal', 'ibm', 'dmc', 'public']:
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    AddToVirtual(virtual[val], files = [('Raw','temp/README')])

# Mutations

## Somatic

In [63]:
mutations = pd.read_csv("temp/wes_somatic_mutations_withlegacy_"+samplesetname+".csv")
damaging = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv', index_col=0)
othercons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv', index_col=0)
othernoncons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv', index_col=0)
hotspot = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv', index_col=0)
hotspot=hotspot.astype(int)
damaging=damaging.astype(int)
othercons=othercons.astype(int)
othernoncons=othernoncons.astype(int)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,22,25,28,29,31,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
#reverting to previous versions
mutations = mutations[mutations.is_likely_immortalization!=True]
mutations = mutations[['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Allele', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'DepMap_ID', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious',
       'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt',
       'ExAC_AF',"Variant_annotation", 'CGA_WES_AC', 'HC_AC',
       'RD_AC', 'RNAseq_AC', 'SangerWES_AC', 'WGS_AC']].rename(columns={"Tumor_Allele":"Tumor_Seq_Allele1"})

In [16]:
! ls temp/README

temp/README


In [105]:
"ACH-001502" in prevmut["ibm"]

True

In [65]:
blacklist = set()
for val in ['internal', 'ibm', "dmc", "public"]:
    print('_________________________________________________')
    print(val)
    missing = set(new[val]) - set(mutations.DepMap_ID)
    print('nott present')
    removed = set(prev[val]) - set(mutations.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevmut[val]) - set(mutations.DepMap_ID)
    print(removed)
    blacklist = (set(mutations.DepMap_ID) - (prevmut[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(blacklist)
    # adding files
    a = mutations[~mutations.DepMap_ID.isin(blacklist)]
    print(len(mutations)- len(a))
    a.to_csv('temp/all_somatic_mutations_withlegacy.csv', index=False)
    a = damaging[~damaging.index.isin(blacklist)]
    print(len(damaging) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
    a = othercons[~othercons.index.isin(blacklist)]
    print(len(othercons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
    a = othernoncons[~othernoncons.index.isin(blacklist)]
    print(len(othernoncons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
    a = hotspot[~hotspot.index.isin(blacklist)]
    print(len(hotspot) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    
    # updating on taiga
    tc.update_dataset(dataset_permaname=taiga_mutation[val],
                     upload_file_path_dict={
    'temp/all_somatic_mutations_withlegacy.csv': 'TableCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv': 'NumericMatrixCSV',
    'temp/README': 'Raw'},
                     dataset_description="""
    #Mutations Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLISTED:
    """+str(blacklist))

    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_mutation[val], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot'), ('README','README')])

_________________________________________________
internal
nott present
{'ACH-001175', 'ACH-001037', 'ACH-001198', 'ACH-002378', 'ACH-002341', 'ACH-001225', 'ACH-001171', 'ACH-002315', 'ACH-002335', 'ACH-001079', 'ACH-001045', 'ACH-001088', 'ACH-001429', 'ACH-001087', 'ACH-001712', 'ACH-001071', 'ACH-001316', 'ACH-001393', 'ACH-001224', 'ACH-001015', 'ACH-001018', 'ACH-001000', 'ACH-001101', 'ACH-002303', 'ACH-001741', 'ACH-001017', 'ACH-001743', 'ACH-001189', 'ACH-001249', 'ACH-001121', 'ACH-002359'}
removed
{'ACH-002315', 'ACH-001189', 'ACH-002378', 'ACH-002341', 'ACH-002359'}
missing
set()
blacklist
{'ACH-001691', 'ACH-002828', 'ACH-001983', 'ACH-001978', 'ACH-001986', 'ACH-001708', 'ACH-002818', 'ACH-001976', 'ACH-001683', 'ACH-001977', 'ACH-001981', 'ACH-001686', 'ACH-001512', 'ACH-002080', 'ACH-001759', 'ACH-002504', 'ACH-001975', 'ACH-002825', 'ACH-001980', 'ACH-001701', 'ACH-002851', 'ACH-001705', 'ACH-001813', 'ACH-001492', 'ACH-001434', 'ACH-001974', 'ACH-002013', 'ACH-001984


Dataset version with id 7b5de4830b9143729456bbc060a2bbd1 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/7b5de4830b9143729456bbc060a2bbd1
[('CCLE_mutations', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_withlegacy'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_damaging'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_othernoncons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_othercons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_hotspot'), ('README', 'depmap-mutation-calls-9be3.36/README')]
hitting https://cds.team/taiga/api/datafile/c90aa3040856420daac

hitting https://cds.team/taiga/api/datafile/6cd93f556ba24e46a73ad8e2754f619a
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 750
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1500
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 U

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: all_somatic_mutations_withlegacy properly converted and uploaded
Uploading all_somatic_mutations_boolmatrix_fordepmap_damaging...
hitting https://cds.team/taiga/api/datafile/07c1897cdf19481f8e469606947faebb
Conversion and upload...:
	 

59311
93
0
94
91
Uploading all_somatic_mutations_withlegacy...
hitting https://cds.team/taiga/api/datafile/581e3dd77d674dd2aeef999425a53977
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: all_somatic_mutations_withlegacy properly converted and uploaded
Uploading all_somatic_mutations_boolmatrix_fordepmap_damag

hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c
hitting https://cds.team/taiga/api/datafile/9a4674254cc64d8fb40bcc6de67caa9c

In [66]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_mutation['internal'], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot')])

[('CCLE_mutations', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_withlegacy'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_damaging'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_othernoncons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_othercons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'depmap-mutation-calls-9be3.36/all_somatic_mutations_boolmatrix_fordepmap_hotspot')]
hitting https://cds.team/taiga/api/datafile/59f4074ce9c24fe793f7b5ac1bde2f64
hitting https://cds.team/taiga/api/datafile/59f4074ce9c24fe793f7b5ac1bde2f64
hitting https://cds.team/taiga/api/datafile/59f4074ce9c24fe793f7b5ac1bde2f64
hitting https://cds.team/taiga/api/datafile/59f4074ce9c24fe793f7b5ac1bde2f64
hitting h

# Copy Number

In [8]:
genecn= pd.read_csv('temp/all_'+samplesetname+'_gene_cn.csv',index_col=0)
segmentcn = pd.read_csv('temp/all_'+samplesetname+'_segment.csv')

In [241]:
#genecn = genecn.apply(lambda x: (x**2)-1)

In [29]:
blacklist = set()
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(segmentcn.DepMap_ID)
    blacklist = (set(segmentcn.DepMap_ID) - (prevcn[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## for segment removing first blacklisted, then embargoed, to create two datasets
    print(len(segmentcn))
    a = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/all_merged_segments.csv', index=False)
    print(len(genecn))
    a = genecn[~genecn.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/all_merged_genes_cn.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # Add to Taiga
    tc.update_dataset(dataset_permaname=taiga_copynumber[val], 
                      upload_file_path_dict={
                        'temp/all_merged_genes_cn.csv': 'NumericMatrixCSV',
                        'temp/all_merged_segments.csv': 'TableCSV',
                        'temp/README': 'Raw'},
                      dataset_description=
    """
    #Copy Number Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLIST:
    """+str(blacklist))
    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_copynumber[val], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn','all_merged_segments')])

_________________________________________________
internal
not present
{'ACH-002315', 'ACH-001189', 'ACH-002303', 'ACH-002341', 'ACH-002359'}
removed
{'ACH-002315', 'ACH-001189', 'ACH-002303', 'ACH-002341', 'ACH-002359'}
missing
set()
blacklist
81 {'ACH-001983', 'ACH-002828', 'ACH-001978', 'ACH-002014', 'ACH-001854', 'ACH-002512', 'ACH-001981', 'ACH-001693', 'ACH-001449', 'ACH-001210', 'ACH-002010', 'ACH-001855', 'ACH-001512', 'ACH-002080', 'ACH-001759', 'ACH-002504', 'ACH-001493', 'ACH-001438', 'ACH-001537', 'ACH-001696', 'ACH-001974', 'ACH-001984', 'ACH-001985', 'ACH-001447', 'ACH-001676', 'ACH-003210', 'ACH-001662', 'ACH-001046', 'ACH-002074', 'ACH-001227', 'ACH-002822', 'ACH-001758', 'ACH-002476', 'ACH-001817', 'ACH-001697', 'ACH-002138', 'ACH-000010', 'ACH-002060', 'ACH-001756', 'ACH-001672', 'ACH-001691', 'ACH-001971', 'ACH-001986', 'ACH-001708', 'ACH-002818', 'ACH-002392', 'ACH-001683', 'ACH-001349', 'ACH-001976', 'ACH-001293', 'ACH-001977', 'ACH-001686', 'ACH-001975', 'ACH-0028

740978
1846
1765
Uploading all_merged_genes_cn...
hitting https://cds.team/taiga/api/datafile/d086b8cb8581490a96955d1a73808e7a
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, l

	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, l

	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: all_merged_genes_cn properly converted and uploaded
Uploading all_merged_segments...
hitting https://cds.team/taiga/api/datafile/6f25509e229543b4babfbaf2576f56cc
Conversion and upload...:
	 Uploading

In [30]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_copynumber['internal'], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn', 'all_merged_segments')])

[('CCLE_gene_cn', 'depmap-wes-cn-data-81a7.35/all_merged_genes_cn'), ('CCLE_segment_cn', 'depmap-wes-cn-data-81a7.35/all_merged_segments')]
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https://cds.team/taiga/api/datafile/44b3c702a1ad4fbb9b462936c0687f1e
hitting https

In [28]:
ls

additionals/                Mutations_CCLE.ipynb
ccle_tasks/                 Mutations_CCLF.ipynb
CDS-NbGq6q.candidateSV.vcf  Mutations_prev_new_comparison.ipynb
CNV_pipeline/               preprocessing_Achilles.ipynb
Copy_Number_CCLE.ipynb      README.md
Copy_Number_CCLF.ipynb      readmes/
data/                       requirements.txt
DepMap Upload-Copy1.ipynb   RNA_CCLE.ipynb
DepMap Upload-Copy2.ipynb   RNA_CCLF.ipynb
DepMap Upload.ipynb         RNA_pipeline/
documentation/              src/
firecloud_documentation.md  temp/
legacy/                     Untitled.ipynb
mutation_pipeline/          WGS_CCLE.ipynb


# RNA

In [196]:
transcripts_tpm = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_tpm.csv',index_col=0)
genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_genes_tpm.csv',index_col=0)
genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_genes_expected_count.csv',index_col=0)
proteincoding_genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_expected_count.csv',index_col=0)
proteincoding_genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_tpm.csv',index_col=0)
transcripts_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_expected_count.csv',index_col=0)

In [197]:
#putting it back to what it was before

# log transforming tpm data
transcripts_tpm=transcripts_tpm.apply(lambda x: np.log2(x+1))
genes_tpm=genes_tpm.apply(lambda x: np.log2(x+1))
proteincoding_genes_tpm=proteincoding_genes_tpm.apply(lambda x: np.log2(x+1))

In [198]:
%store -r failed

no stored variable or alias rnafailed


In [202]:
blacklist = set()
for val in ['internal','ibm', 'dmc','public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(genes_tpm.index)
    print(removed)
    print('removed for QC reasons')
    print(failed)
    print('removed')
    removed = set(prevrna[val]) - set(genes_tpm.index)
    #print(removed - set(rename.keys()))
    missing = set(new[val]) - set(genes_tpm.index)
    blacklist = (set(genes_tpm.index) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)

    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(genes_expected_count))
    a = genes_expected_count[~genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_expected_count.csv')
    print(len(genes_tpm))
    a = genes_tpm[~genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_tpm.csv')
    print(len(proteincoding_genes_tpm))
    a = proteincoding_genes_tpm[~proteincoding_genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_tpm.csv')
    print(len(transcripts_tpm))
    a = transcripts_tpm[~transcripts_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_tpm.csv')
    print(len(proteincoding_genes_expected_count))
    a = proteincoding_genes_expected_count[~proteincoding_genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_expected_count.csv')
    print(len(transcripts_expected_count))
    a = transcripts_expected_count[~transcripts_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_expected_count.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # adding to taiga
    tc.update_dataset(dataset_permaname=taiga_expression[val],
                     upload_file_path_dict={
                       'temp/expression_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_expected_count.csv': 'NumericMatrixCSV',
                     "temp/README": "Raw"},
                      dataset_description=
    """
    # INTERNAL RNA

    for information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str(failed)+"""

    BLACKLIST:
    """+str(blacklist))

    # add to virtual 
    AddToVirtual(virtual[val], taiga_expression[val], files=[
    ('CCLE_expression_full', 'expression_genes_tpm'), 
    ('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
    ('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
    ('CCLE_expression', 'expression_proteincoding_genes_tpm'),
    ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),
    ('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

_________________________________________________
internal
not present
{'ACH-002162', 'ACH-001662', 'ACH-001101', 'ACH-002116', 'ACH-002152', 'ACH-002247', 'ACH-002221', 'ACH-002105', 'ACH-001091', 'ACH-002220', 'ACH-001044', 'ACH-001047', 'ACH-002100', 'ACH-001384', 'ACH-001224', 'ACH-002316', 'ACH-002021', 'ACH-002185', 'ACH-002356', 'ACH-002374', 'ACH-002348', 'ACH-002106', 'ACH-002289', 'ACH-002096', 'ACH-002224', 'ACH-002341', 'ACH-002394', 'ACH-001137', 'ACH-002344', 'ACH-001130', 'ACH-002092', 'ACH-002400', 'ACH-002161', 'ACH-001678', 'ACH-001233', 'ACH-002155', 'ACH-001364', 'ACH-002258', 'ACH-002243', 'ACH-002458', 'ACH-002306', 'ACH-001605', 'ACH-002300', 'ACH-001704', 'ACH-001092', 'ACH-001093', 'ACH-002168', 'ACH-002357', 'ACH-002465', 'ACH-001528', 'ACH-002244', 'ACH-001182', 'ACH-002304', 'ACH-002313', 'ACH-002122', 'ACH-001107', 'ACH-002358', 'ACH-002359', 'ACH-002192', 'ACH-001547', 'ACH-001649', 'ACH-002298', 'ACH-002040', 'ACH-002178', 'ACH-000047', 'ACH-002099', 'ACH

KeyboardInterrupt: 

In [89]:
AddToVirtual('depmap-a0ab', taiga_expression['internal'], files=[
('CCLE_expression_full', 'expression_genes_tpm'), 
('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
('CCLE_expression', 'expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

[('CCLE_expression_full', 'depmap-rnaseq-expression-data-363a.36/expression_genes_tpm'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-363a.36/expression_transcripts_tpm'), ('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-363a.36/expression_genes_expected_count'), ('CCLE_expression', 'depmap-rnaseq-expression-data-363a.36/expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'depmap-rnaseq-expression-data-363a.36/expression_proteincoding_genes_expected_count'), ('CCLE_expression_transcripts_expected_count', 'depmap-rnaseq-expression-data-363a.36/expression_transcripts_expected_count')]
hitting https://cds.team/taiga/api/datafile/982a8b4e984b4b9fbf91d54d3fb4cc6d
hitting https://cds.team/taiga/api/datafile/982a8b4e984b4b9fbf91d54d3fb4cc6d
hitting https://cds.team/taiga/api/datafile/982a8b4e984b4b9fbf91d54d3fb4cc6d
hitting https://cds.team/taiga/api/datafile/982a8b4e984b4b9fbf91d54d3fb4cc6d
hitting https://cds.team/taiga/api/datafile

# Fusions

In [84]:
fusions = pd.read_csv('temp/fusions_'+samplesetname+'.csv')
filtered = pd.read_csv('temp/filtered_fusions_'+samplesetname+'.csv')

In [ ]:
%store -r failed

In [94]:
print(set(fusions.DepMap_ID) - set(filtered.DepMap_ID))
print(set(filtered.DepMap_ID) - set(fusions.DepMap_ID))

set()
set()


In [88]:
blacklist = set()
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(fusions.DepMap_ID)
    print(removed)
    print('removed for QC reasons')
    print(failed)
    print('removed')
    removed = set(prevrna[val]) - set(fusions.DepMap_ID)
    print(removed - set(rename.keys()))
    missing = set(new[val]) - set(fusions.DepMap_ID)
    blacklist = (set(fusions.DepMap_ID) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(fusions))
    a = fusions[~fusions.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/fusions.csv', index=False)
    print(len(filtered))
    a= filtered[~filtered.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/filtered_fusions.csv', index=False)

    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # uploading to taiga
    tc.update_dataset(dataset_permaname=taiga_fusion[val],
                     upload_file_path_dict={
                         'temp/fusions.csv': 'TableCSV',
                         'temp/filtered_fusions.csv': 'TableCSV',
                         'temp/README': 'Raw'},
                      dataset_description=
    """
    # Internal Fusions

    for more information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str(failed)+"""

    BLACKLIST:
    """+str(blacklist))

    # virtual datasets
    AddToVirtual(virtual[val], taiga_fusion[val], files=[('CCLE_fusions_unfiltered', 'fusions'),('CCLE_fusions', 'filtered_fusions')])

_________________________________________________
internal
not present
{'ACH-002304', 'ACH-001489', 'ACH-002133', 'ACH-002144', 'ACH-001002', 'ACH-002245', 'ACH-001107', 'ACH-002191', 'ACH-002093', 'ACH-002252', 'ACH-002105', 'ACH-002175', 'ACH-002346', 'ACH-002097', 'ACH-002204', 'ACH-002161', 'ACH-001143', 'ACH-002190', 'ACH-002192', 'ACH-002337', 'ACH-001639', 'ACH-002376', 'ACH-001130', 'ACH-002122', 'ACH-002180', 'ACH-001125', 'ACH-002291', 'ACH-002285', 'ACH-002089', 'ACH-002316', 'ACH-002387', 'ACH-001171', 'ACH-002315', 'ACH-002241', 'ACH-002173', 'ACH-001955', 'ACH-002312', 'ACH-002142', 'ACH-001350', 'ACH-002292', 'ACH-002141', 'ACH-001063', 'ACH-002389', 'ACH-002382', 'ACH-002279', 'ACH-001233', 'ACH-001064', 'ACH-002248', 'ACH-001502', 'ACH-002107', 'ACH-001137', 'ACH-001249', 'ACH-002178', 'ACH-001108', 'ACH-002183', 'ACH-002260', 'ACH-001047', 'ACH-002121', 'ACH-002340', 'ACH-002102', 'ACH-000992', 'ACH-002395', 'ACH-001138', 'ACH-001234', 'ACH-002176', 'ACH-002143', 'ACH

hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datafile/833379f114c54ecda9ec0f73a2ed2370
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 9

	 Uploading to S3
	 Uploading to S3

	 Done: filtered_fusions properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/cd0b98dd5c9b45bbb386a0809388bcf9
Conversion and upload...:
	 Uploading the DataFileFormat.Raw to S3

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 85aaa68f08c94e5b94a39408480c8f75 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/85aaa68f08c94e5b94a39408480c8f75
[('CCLE_fusions_unfiltered', 'fusion-dc08.5/fusions'), ('CCLE_fusions', 'fusion-dc08.5/filtered_fusions')]
hitting https://cds.team/taiga/api/datafile/c18a6f78a1e74d22b78478db33a786be
hitting https://cds.team/taiga/api/datafile/c18a6f78a1e74d22b78478db33a786be
hitting https://cds.team/taiga/api/datafile/c18a6f78a1e74d22b78478db33a786be
hitting https://cds.team/taiga/api/datafile/c18a6f78a1e74d22b78478db33a786be
hitting https://cds.team/ta

356396
44802
44057
Uploading fusions...
hitting https://cds.team/taiga/api/datafile/a6f8e263ff904f529dd77ea151b26a4d
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: fusions properly converted and uploaded
Uploading filtered_fusions...
hitting https://cds.team/taiga/api/datafile/a6f8e263ff904f529dd77ea151b26a4d
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3

	 Done: filtered_fusions properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/a6f8e263ff904f529dd77ea151b26a4d
Conversion and upload...:
	 Uploading the DataFileFormat.Raw to S3

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVer

44802
43388
Uploading fusions...
hitting https://cds.team/taiga/api/datafile/91fbefe314124aa8930faf6b9350a7ee
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: fusions properly converted and uploaded
Uploading filtered_fusions...
hitting https://cds.team/taiga/api/datafile/91fbefe314124aa8930faf6b9350a7ee
Conversion and upload...:
	 Uploading to S3

	 Done: filtered_fusions properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/91fbefe314124aa8930faf6b9350a7ee
Conversion and upload...:
	 Waiting in the task queue

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id ffb573a74d2a41d0afb076241f86aee2 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/ffb573a7

In [90]:
AddToVirtual('depmap-a0ab', "gene-fusions-8b7a", files=[('CCLE_fusions_unfiltered', 'fusions'),('CCLE_fusions', 'filtered_fusions')])

[('CCLE_fusions_unfiltered', 'gene-fusions-8b7a.20/fusions'), ('CCLE_fusions', 'gene-fusions-8b7a.20/filtered_fusions')]
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/api/datafile/dca7ae8e8142431897ee0d8e4e4377ed
hitting https://cds.team/taiga/a